# Why Twilight? Proving WFD/DDF Saturation for Nearby SNe Ia

Using Rubin/LSST’s standard SNR/zeropoint formalism, many **low-z SNe Ia** saturate **per read** in normal **WFD** and **DDF** observations near peak light (especially in g/r/i). ([rubinobservatory.org][1], [smtn-002.lsst.io][2])

---

## What this notebook demonstrates

* Computes **peak-pixel saturation magnitudes** $m_{\rm sat}$ in ugrizy from `photom_rubin.py` (Rubin-style ZP, PSF NEA, sky electrons/pixel). ([smtn-002.lsst.io][2])
* Maps $m_{\rm sat}$ → **typical redshift** for a standard SN Ia at peak using flat ΛCDM.
* Compares documented **survey plan variants**:
  **WFD** 1×30 s (current framing), legacy **2×15 s**, **Phase-3** tweak (**u = 38 s**, others ≈ 29 s), and **DDF** with better seeing and 30 s reads. ([rubinobservatory.org][1], [pstn-054.lsst.io][3], [Observing Strategy][4], [pstn-056.lsst.io][5])

---

## Key assumptions (Rubin conventions)

* **Zeropoints & SNR framework:** SMTN-002 (1-s AB zeropoints, extinction via $k_m(X-1)$, NEA for Gaussian PSF). **Gain** can be assumed 1 e⁻/ADU for SNR/m₅ calculations—consistent with SMTN-002. ([smtn-002.lsst.io][2])
* **Survey exposure styles:**
  - **WFD** is commonly described as \~**30 s per visit**; historical split **2×15 s** appears in earlier baselines and docs. **Phase-3** recommendations adjust **u** to **\~38 s** and reduce other bands to **\~29 s** in simulations (v3.5+). ([rubinobservatory.org][1], [pstn-054.lsst.io][3], [Observing Strategy][4], [pstn-056.lsst.io][5])
  - **DDF** consists of sequences of many \~**30 s** visits with typically **better seeing**, which concentrates flux and makes saturation more likely for the same exposure time. ([Observing Strategy][6])
* **Detector headroom:** We treat **100 ke⁻/pixel** as a conservative hard cap and **80–100 ke⁻** as a non-linearity warning band.
* **Cosmology:** Flat ΛCDM, $H_0=70~{\rm km\,s^{-1}\,Mpc^{-1}},~\Omega_m=0.3$.
* **Sky:** Rubin dark-sky surface brightness per band (SMTN-002 style) used for non-twilight comparisons. ([smtn-002.lsst.io][2])

---

## Method (what the code does)

We compute the **per-read** saturation limit by solving the **peak-pixel** charge budget:

1. **Zeropoint in electrons per exposure**

$$
ZPT_{pe} \;=\; ZPT^{(1{\rm s})} \;+\; 2.5\log_{10}(t_{\rm exp}) \;-\; k_m\,(X-1).
$$

2. **Central-pixel fraction** for a 2D Gaussian PSF sampled on square pixels:

$$
f_{\rm cen} \;=\; \big[\,\mathrm{erf}\big(\tfrac{p}{2\sqrt{2}\,\sigma}\big)\big]^2,\quad \sigma=\frac{{\rm FWHM}}{2.355},\ \ p=\text{pixel scale}.
$$

3. **Sky electrons per pixel** via your pipeline’s `SKYSIG`:

$$
N_{\rm sky,pix} \;=\; ({\rm SKYSIG}\times {\rm GAIN})^2 \;=\; 10^{-0.4\,(m_{\rm sky}-ZPT_{pe})}\,p^2.
$$

4. **Saturation condition** (solve for the apparent magnitude that just hits the cap):

$$
f_{\rm cen}\,10^{-0.4\,(m_{\rm sat}-ZPT_{pe})} \;+\; N_{\rm sky,pix} \;=\; N_{\rm sat}.
$$

5. **Closed-form solution**

$$
m_{\rm sat} \;=\; ZPT_{pe} \;-\; 2.5\log_{10}\!\left(\frac{N_{\rm sat}-N_{\rm sky,pix}}{f_{\rm cen}}\right).
$$

All of these are exactly the scalars your `photom_rubin.py` computes or wraps, following Rubin conventions. ([smtn-002.lsst.io][2])

---

## Survey plan variants compared (per-read exposures)

* **WFD 1×30 s** (current framing used in Rubin 101, alerts/brokers pages). ([rubinobservatory.org][1])
* **WFD 2×15 s** (legacy baseline; still informative because **per-read** saturation is then set by 15 s). ([pstn-054.lsst.io][3])
* **WFD Phase-3 tweak:** **u = 38 s**, other bands **≈ 29 s** (implemented in recent v3.5+ simulations). ([Observing Strategy][4], [pstn-056.lsst.io][5], [Rubin Observatory LSST Community forum][7])
* **DDF 1×30 s, better seeing** (same exposure, sharper PSF → lower $m_{\rm sat}$). ([Observing Strategy][6])

> **note:** Coadds do **not** raise the per-read pixel-well limit; saturation is a **per exposure** detector effect.

---

## Caveats & extensions

* **K-corrections & extinction** are turned **off by default** (reasonable for $z\lesssim 0.05$); enable if you want band-by-band realism.
* **PSF shape**: a Gaussian core is conservative; a sharper core (e.g., with non-Gaussian wings) can **lower** $m_{\rm sat}$ further.
* **Brighter-fatter & nonlinearity**: your **80–100 ke⁻** warning band accounts for the onset of nonlinearity; stays conservative.
* **Host surface brightness**: at these bright levels the **source** dominates the peak-pixel term; a bright bulge (μ ≈ 18 mag/arcsec²) shifts $m_{\rm sat}$ by only a few 0.1 mag in r for 15–30 s reads (you can include μ\_host in the sky term if desired).

---

## References (Rubin/LSST)

* **SMTN-002: Calculating LSST limiting magnitudes and SNR** — formalism used here (ZP_1s, SNR, gain≈1, m₅ recipe). ([smtn-002.lsst.io][2])
* **Rubin 101 — WFD & DDF overview** — “\~30-s exposure every \~3 days” (WFD) & DDF description. ([rubinobservatory.org][1])
* **PSTN-054 (2022)** — baseline assumption of **30 s total = 2×15 s** snaps in performance estimates (historical context). ([pstn-054.lsst.io][3])
* **Survey-strategy release notes (v3.5)** — Phase-3 exposure updates (**u = 38 s**, others **≈ 29 s**). ([Observing Strategy][4])
* **PSTN-056 (2025)** — SCOC Phase-3 recommendations; exposure time adjustments and rolling cadence notes. ([pstn-056.lsst.io][5])
* **DDF baseline page** — sequences of many visits, share of time, and practice of deeper coadds (per-read exposure remains \~30 s). ([Observing Strategy][6])
* **Community discussion: 2×15 s → 1×30 s** — scientific and technical trade-offs. ([Rubin Observatory LSST Community forum][8])

---

[1]: https://rubinobservatory.org/for-scientists/rubin-101/the-legacy-survey-of-space-and-time-lsst "The Legacy Survey of Space and Time (LSST)"
[2]: https://smtn-002.lsst.io/ "Calculating LSST limiting magnitudes and SNR"
[3]: https://pstn-054.lsst.io/PSTN-054.pdf "PSTN-054.pdf"
[4]: https://survey-strategy.lsst.io/baseline/changes.html "Updates to the baseline — Observing Strategy"
[5]: https://pstn-056.lsst.io/PSTN-056.pdf "Vera C. Rubin Observatory Project Science Team"
[6]: https://survey-strategy.lsst.io/baseline "Deep Drilling Fields — Observing Strategy"
[7]: https://community.lsst.org/t/public-scoc-meeting-minutes/7185?page=2 "Public SCOC meeting minutes - Survey Strategy"
[8]: https://community.lsst.org/t/scientific-impact-of-moving-from-2-snaps-to-a-single-exposure/3266 "Scientific impact of moving from 2 snaps to a single exposure"

In [41]:
import math
from typing import Dict
import pandas as pd
from astropy.cosmology import FlatLambdaCDM, z_at_value
import astropy.units as u
from twilight_planner_pkg.photom_rubin import (
    PhotomConfig,
    epoch_zeropoints,
    sky_rms_adu_per_pix,
    central_pixel_fraction_gaussian,
)

#  Dark-sky surface brightness (mag/arcsec^2)
DARK_SKY = {
    "u": 22.95,
    "g": 22.24,
    "r": 21.20,
    "i": 20.46,
    "z": 19.60,
    "y": 18.61,
}

bands = ["u", "g", "r", "i", "z", "y"]
X = 1.2  # airmass for zeropoints
cfg = PhotomConfig()

#seeing assume:
WFD_MEDIAN = {"u": 0.90, "g": 0.85, "r": 0.83, "i": 0.83, "z": 0.85, "y": 0.90}
DDF_BETTER = {"u": 0.80, "g": 0.70, "r": 0.60, "i": 0.60, "z": 0.65, "y": 0.70}

* **WFD 1×30 s** (current framing used in Rubin 101, alerts/brokers pages).
* **WFD 2×15 s** (legacy baseline; still informative because **per-read** saturation is then set by 15 s).
* **WFD Phase-3 tweak:** **u = 38 s**, other bands **≈ 29 s** (implemented in recent v3.5+ simulations).
* **DDF 1×30 s, better seeing** (same exposure, sharper PSF → lower $m_{\rm sat}$).

In [42]:
# Scenarios
SCENARIOS: Dict[str, dict] = {
    # Rubin “~30 s per visit”  for WFD
    "WFD_1x30": {
        "exp_s": {b: 30.0 for b in bands},
        "fwhm": WFD_MEDIAN,
        "note": "WFD single 30 s exposure per visit; baseline style.",
    },
    # 2×15 s snaps (per-read saturation uses 15 s)
    "WFD_2x15_legacy": {
        "exp_s": {b: 15.0 for b in bands},
        "fwhm": WFD_MEDIAN,
        "note": "Legacy 2×15 s snaps; per-read saturation at 15 s.",
    },
    # SCOC recommendation: u=38 s, others shortened by ~0.8 s
    "WFD_u38_others29p2": {
        "exp_s": {"u": 38.0, **{b: 29.2 for b in bands if b != "u"}},
        "fwhm": WFD_MEDIAN,
        "note": "SCOC rec: u=38 s; others≈29.2 s to balance survey time.",
    },
    # Community-tested variant: grizy=20 s, u=40 s (pontus-2489)
    "WFD_grizy20_u40": {
        "exp_s": {"u": 40.0, **{b: 20.0 for b in bands if b != "u"}},
        "fwhm": WFD_MEDIAN,
        "note": "Variant tested in cadence studies: grizy=20 s, u=40 s.",
    },
    # DDF per-read exposure typically 30 s; better seeing
    "DDF_1x30_betterSeeing": {
        "exp_s": {b: 30.0 for b in bands},
        "fwhm": DDF_BETTER,
        "note": "DDF per-visit 30 s with sub-arcsecond seeing.",
    },
    # If DDF with 2×15 reads:
    "DDF_2x15_betterSeeing": {
        "exp_s": {b: 15.0 for b in bands},
        "fwhm": DDF_BETTER,
        "note": "DDF legacy-style 2×15 s; per-read saturation at 15 s.",
    },
}

In [43]:
# Peak-pixel saturation magnitude
def sat_mag(band: str, t_exp_s: float, fwhm_eff_arcsec: float) -> float:
    # Zeropoint in e- for t_exp_s at airmass X
    ZPT_pe, _ = epoch_zeropoints(cfg.zpt1s[band], t_exp_s, cfg.k_m[band], X, cfg.gain_e_per_adu)

    # Central-pixel fraction for Gaussian PSF on square pixels
    f_cen = central_pixel_fraction_gaussian(fwhm_eff_arcsec, cfg.pixel_scale_arcsec)

    # Sky electrons per pixel via SKYSIG: (SKYSIG * GAIN)^2
    SKYSIG = sky_rms_adu_per_pix(DARK_SKY[band], ZPT_pe, cfg.pixel_scale_arcsec, cfg.gain_e_per_adu)
    N_sky_pix = (SKYSIG * cfg.gain_e_per_adu) ** 2

    # Solve: f_cen * 10^(-0.4*(m - ZPT_pe)) + N_sky_pix = N_sat
    N_sat = cfg.npe_pixel_saturate
    val = max(1e-9, (N_sat - N_sky_pix) / max(1e-12, f_cen))

    return ZPT_pe - 2.5 * math.log10(val)

def saturation_table(exp_s: Dict[str, float], fwhm: Dict[str, float]) -> pd.DataFrame:
    rows = []
    for b in ["g", "r", "i", "z", "y"]:
        m = sat_mag(b, exp_s[b], fwhm[b])
        rows.append({"band": b, "t_exp_s": exp_s[b], "FWHM\":": fwhm[b], "m_sat": round(m, 2)})
    return pd.DataFrame(rows).sort_values("band").reset_index(drop=True)

In [44]:
# SN Ia “typical” redshift from saturation magnitude
ABS_MAG = {"g": -19.1, "r": -19.2, "i": -19.3, "z": -19.3, "y": -19.2}
K_CORR = {b: 0.0 for b in ABS_MAG}   # set to 0 for z≲0.05; tweak if desired
A_EXT  = {b: 0.0 for b in ABS_MAG}   # host+MW extinction (mag)
dM_intrinsic = 0.2                   # ±0.2 mag intrinsic scatter

cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)

def z_from_mu(mu: float) -> float:
    dL = 10 ** ((mu - 25.0) / 5.0) * u.Mpc
    return float(z_at_value(cosmo.luminosity_distance, dL, zmin=1e-6, zmax=1.0))

def add_redshift_columns(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    z_typ, z_lo, z_hi = [], [], []
    for _, row in out.iterrows():
        b, m_sat = row["band"], float(row["m_sat"])
        mu = m_sat - ABS_MAG[b] - K_CORR[b] - A_EXT[b]
        z  = z_from_mu(mu)
        zL = z_from_mu(mu - dM_intrinsic)
        zH = z_from_mu(mu + dM_intrinsic)
        z_typ.append(z); z_lo.append(zL); z_hi.append(zH)
    out["z_typical"] = z_typ
    out[f"z_low(M+{dM_intrinsic:.1f})"] = z_lo
    out[f"z_high(M-{dM_intrinsic:.1f})"] = z_hi
    return out

In [45]:
# Run all scenarios
all_results: Dict[str, pd.DataFrame] = {}
globals().update(all_results)

for name, spec in SCENARIOS.items():
    base = saturation_table(spec["exp_s"], spec["fwhm"])
    all_results[name] = add_redshift_columns(base)

for name, df in all_results.items():
    globals()[name] = df
    print(f"\n=== {name} ===")
    print(df.to_string(index=False))


scenario_dfs = {name: df.copy() for name, df in all_results.items()}


=== WFD_1x30 ===
band  t_exp_s  FWHM":  m_sat  z_typical  z_low(M+0.2)  z_high(M-0.2)
   g     30.0    0.85  16.37   0.028374      0.025925       0.031050
   i     30.0    0.83  16.11   0.027617      0.025231       0.030222
   r     30.0    0.83  16.29   0.028631      0.026160       0.031331
   y     30.0    0.90  14.60   0.013300      0.012140       0.014569
   z     30.0    0.85  15.68   0.022738      0.020768       0.024892

=== WFD_2x15_legacy ===
band  t_exp_s  FWHM":  m_sat  z_typical  z_low(M+0.2)  z_high(M-0.2)
   g     15.0    0.85  15.61   0.020119      0.018373       0.022028
   i     15.0    0.83  15.35   0.019579      0.017879       0.021438
   r     15.0    0.83  15.53   0.020302      0.018541       0.022229
   y     15.0    0.90  13.83   0.009357      0.008539       0.010253
   z     15.0    0.85  14.92   0.016104      0.014702       0.017637

=== WFD_u38_others29p2 ===
band  t_exp_s  FWHM":  m_sat  z_typical  z_low(M+0.2)  z_high(M-0.2)
   g     29.2    0.85  16.34   0


## Reading the results

* In **WFD 30 s**, peak-pixel saturation typically occurs near **g ≈ 16.4, r ≈ 16.3, i ≈ 16.1, z ≈ 15.7, y ≈ 14.6** under median seeing—so $z\sim0.02\!-\!0.03$ SNe Ia at peak often saturate. **DDF** saturates at **fainter** total magnitudes for the same 30 s read (even more low-z SNe affected).
* Short **twilight** reads (e.g., 5–15 s) raise $m_{\rm sat}$ by $\Delta m \approx 2.5\log_{10}(t_2/t_1)$ → **0.75–1.75 mag** for factors of 2–8 shorter, which is exactly why twilight buys unsaturated photometry at low z.

In [46]:
WFD_1x30

,band,t_exp_s,"FWHM"":",m_sat,z_typical,z_low(M+0.2),z_high(M-0.2)
0,g,30.0,0.85,16.37,0.028374,0.025925,0.031050
1,i,30.0,0.83,16.11,0.027617,0.025231,0.030222
2,r,30.0,0.83,16.29,0.028631,0.026160,0.031331
3,y,30.0,0.90,14.60,0.013300,0.012140,0.014569
4,z,30.0,0.85,15.68,0.022738,0.020768,0.024892


In [47]:
WFD_2x15_legacy

,band,t_exp_s,"FWHM"":",m_sat,z_typical,z_low(M+0.2),z_high(M-0.2)
0,g,15.0,0.85,15.61,0.020119,0.018373,0.022028
1,i,15.0,0.83,15.35,0.019579,0.017879,0.021438
2,r,15.0,0.83,15.53,0.020302,0.018541,0.022229
3,y,15.0,0.90,13.83,0.009357,0.008539,0.010253
4,z,15.0,0.85,14.92,0.016104,0.014702,0.017637


In [48]:
WFD_u38_others29p2

,band,t_exp_s,"FWHM"":",m_sat,z_typical,z_low(M+0.2),z_high(M-0.2)
0,g,29.2,0.85,16.34,0.027993,0.025576,0.030633
1,i,29.2,0.83,16.08,0.027245,0.024892,0.029817
2,r,29.2,0.83,16.26,0.028247,0.025808,0.030911
3,y,29.2,0.90,14.57,0.013119,0.011975,0.014371
4,z,29.2,0.85,15.65,0.022431,0.020487,0.024556


In [49]:
WFD_grizy20_u40

,band,t_exp_s,"FWHM"":",m_sat,z_typical,z_low(M+0.2),z_high(M-0.2)
0,g,20.0,0.85,15.93,0.023259,0.021244,0.025460
1,i,20.0,0.83,15.66,0.022533,0.020580,0.024667
2,r,20.0,0.83,15.84,0.023364,0.021341,0.025576
3,y,20.0,0.90,14.15,0.010831,0.009885,0.011866
4,z,20.0,0.85,15.23,0.018541,0.016930,0.020302
